In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
def scrape_flamecomics_chapters(url):
    """
    Scrapes chapter links and their details from a given flamecomics.xyz series URL.

    Args:
        url (str): The URL of the series on flamecomics.xyz.

    Returns:
        list: A list of dictionaries containing chapter details.
    """
    try:
        # Fetch the webpage content
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad HTTP responses
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all chapter elements
        chapters = []
        chapter_elements = soup.find_all('a', class_='ChapterCard_chapterWrapper__j8pBx')

        for chapter in chapter_elements:
            chapter_data = {
                'link': chapter['href'],  # Extract the chapter link
                'title': chapter.find('p', class_='mantine-Text-root').text.strip(),  # Extract the chapter title
                'date': chapter.find_all('p', class_='mantine-Text-root')[1].text.strip(),  # Extract the date
                'reactions': [
                    img['alt'] for img in chapter.find_all('img', class_='ReactionBarMini_reactionIcon__rkqnd')
                ]  # Extract reaction icons
            }
            chapters.append(chapter_data)

        return chapters

    except requests.exceptions.RequestException as e:
        print(f"Error fetching the URL: {e}")
        return []
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

# Example usage
url = "https://flamecomics.xyz/series/2"
chapters = scrape_flamecomics_chapters(url)
for chapter in chapters:
    print(chapter)

Development

In [ ]:
import requests
import re
import os
import json
import zipfile
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO

url = "https://flamecomics.xyz/series/2"
# url = "https://flamecomics.xyz/series/143"

response = requests.get(url)
response.raise_for_status()  # Raise an error for bad HTTP responses
soup = BeautifulSoup(response.text, 'html.parser')

# Find the <script> tag with id="__NEXT_DATA__"
script_tag = soup.find('script', id='__NEXT_DATA__', type='application/json')
if script_tag:
    # Extract the JSON content and parse it into a Python dictionary
    dat = json.loads(script_tag.string)['props']['pageProps']
else:
    print("No series object (__NEXT_DATA__) found.")

series_title = dat['series']['title']

# Create a folder with the series name if it doesn't already exist
dir_series = os.path.join('data', series_title)
if not os.path.exists(dir_series):
    os.makedirs(dir_series)

chapters = dat['chapters']
for chapter in chapters:
    series_id = chapter['series_id']
    chapter_title = chapter['title']
    chapter_number = chapter['chapter']
    chapter_number = chapter_number.split(".")[0] if chapter_number.endswith(".0") else chapter_number
    chapter_token = chapter['token']
    cbz_filename = f"{series_title} - Chapter {chapter_number}.cbz"
    file_cbz = os.path.join(dir_series, cbz_filename)
    if os.path.exists(file_cbz):
        # print(f"{cbz_filename} already exists, skipping download.")
        continue
    # Check if the chapter has images
    if not chapter['images']:
        print(f"No images found for chapter {chapter_number}. Skipping download.")
        continue
    # Check if the chapter has a token
    if not chapter_token:
        print(f"No token found for chapter {chapter_number}. Skipping download.")
        continue
    # Check if the chapter has a series ID
    if not series_id:
        print(f"No series ID found for chapter {chapter_number}. Skipping download.")
        continue
    images = chapter['images']
    imgs = []
    images = dict(sorted(images.items(), key=lambda item: int(item[0])))
    print(f"Downloading: {series_title} - Chapter {chapter_number}")
    for key in images:
        image = images[key]
        img_name = image['name']
        type = image['type'][0]
        url_image = f"https://cdn.flamecomics.xyz/series/{series_id}/{chapter_token}/{img_name}"
        # Download the image
        image_response = requests.get(url_image)
        image_response.raise_for_status()  # Ensure the request was successful
        # Load the image into memory
        img = Image.open(BytesIO(image_response.content))
        # Add image to list
        imgs.append([img, type])

    # Save images as a CBZ file
    with BytesIO() as cbz_buffer:
        # Save all images into the buffer as a ZIP archive
        with zipfile.ZipFile(cbz_buffer, 'w') as cbz:
            for idx, img in enumerate(imgs):
                img_filename = f"{idx + 1:03}.jpg"
                with BytesIO() as img_buffer:
                    if img[1] == 'image/jpeg':
                        img[0].save(img_buffer, format='JPEG')
                    elif img[1] == 'image/png':
                        img[0] = img[0].convert("RGB")
                        img[0].save(img_buffer, format='JPEG')
                    cbz.writestr(img_filename, img_buffer.getvalue())
        # Write the buffer to a file
        with open(file_cbz, 'wb') as cbz_file:
            cbz_file.write(cbz_buffer.getvalue())
    # print(f"{cbz_filename} saved in {dir_series}")

Downloading: Omniscient Reader's Viewpoint - Chapter 156
Downloading: Omniscient Reader's Viewpoint - Chapter 155
Downloading: Omniscient Reader's Viewpoint - Chapter 154
Downloading: Omniscient Reader's Viewpoint - Chapter 153
Downloading: Omniscient Reader's Viewpoint - Chapter 152
Downloading: Omniscient Reader's Viewpoint - Chapter 151
Downloading: Omniscient Reader's Viewpoint - Chapter 150
Downloading: Omniscient Reader's Viewpoint - Chapter 149
Downloading: Omniscient Reader's Viewpoint - Chapter 148
Downloading: Omniscient Reader's Viewpoint - Chapter 147
Downloading: Omniscient Reader's Viewpoint - Chapter 146
Downloading: Omniscient Reader's Viewpoint - Chapter 145
Downloading: Omniscient Reader's Viewpoint - Chapter 144
Downloading: Omniscient Reader's Viewpoint - Chapter 143
Downloading: Omniscient Reader's Viewpoint - Chapter 142
Downloading: Omniscient Reader's Viewpoint - Chapter 141
Downloading: Omniscient Reader's Viewpoint - Chapter 140
Downloading: Omniscient Reader'

In [39]:
chapter = chapters[92]
images = chapter['images']
for key in images:
    print(images[key])

{'size': 849489, 'type': ['image/png'], 'name': 'ORV-credit-page.png', 'modified': '2024-09-08T20:58:58.000Z', 'width': 1778, 'height': 1000}
{'size': 915702, 'type': ['image/jpeg'], 'name': 'OVR165-01-2.jpg', 'modified': '2024-09-08T20:58:59.000Z', 'width': 800, 'height': 12655}
{'size': 962084, 'type': ['image/jpeg'], 'name': 'OVR165-02-2.jpg', 'modified': '2024-09-08T20:59:00.000Z', 'width': 800, 'height': 12800}
{'size': 747868, 'type': ['image/jpeg'], 'name': 'OVR165-03-2.jpg', 'modified': '2024-09-08T20:59:00.000Z', 'width': 800, 'height': 14168}
{'size': 836307, 'type': ['image/jpeg'], 'name': 'OVR165-04-2.jpg', 'modified': '2024-09-08T20:59:00.000Z', 'width': 800, 'height': 13380}
{'size': 702459, 'type': ['image/jpeg'], 'name': 'OVR165-05-2.jpg', 'modified': '2024-09-08T20:59:00.000Z', 'width': 800, 'height': 15072}
{'size': 913142, 'type': ['image/jpeg'], 'name': 'OVR165-06-3.jpg', 'modified': '2024-09-08T20:59:01.000Z', 'width': 800, 'height': 14777}
{'size': 1836982, 'type'

In [ ]:
dat['props']['pageProps']['series']['chapters']

In [8]:
dat['props']['pageProps']['series']['title']

"Omniscient Reader's Viewpoint"

In [16]:
dat['pageProps'].keys()

dict_keys(['series', 'chapters'])

In [6]:
dat['props']

{'cookies': {},
 '__N_SSG': True,
 'pageProps': {'series': {'series_id': 2,
   'title': "Omniscient Reader's Viewpoint",
   'altTitles': '["전지적 독자 시점 "," 전독시 "," Jeonjijeok Dokja Sijeom "," Lecteur omniscient "," Lector omnisciente "," ORV "," Punkt Widzenia Wszechwiedzącego Czytelnika "," Всеведущий читатель "," دیدگاه خواننده\u200cی همه\u200cچی\u200cدون "," وجهة نظر القارئ العراف "," อ่านชะตาวันสิ้นโลก "," 全知的な読者の視点から "," 全知讀者視角 "," 全知读者"]',
   'description': '‘This is a development that I know of.’ The moment he thought that the world had been destroyed, and a new universe had unfolded. The new life of an ordinary reader begins within the world of the novel, a novel that he alone had finished.',
   'language': 'English',
   'type': 'Manhwa',
   'tags': ['Action', 'Adventure', 'Fantasy', 'Survival'],
   'country': 'KR',
   'author': 'Sing-Shong',
   'artist': 'Sleepy-C (REDICE STUDIO)',
   'publisher': 'Naver Webtoon (Naver)',
   'year': 2020,
   'status': 'Ongoing',
   'schedule': '

# APIs
https://flamecomics.xyz/api/series